In [ ]:
"""
RIMAS Web Scraper - Monitoramento de Águas Subterrâneas
Autor: lhazmelo
Data: Junho/2024
Descrição:
    Este script automatiza a coleta de dados de poços de monitoramento do sistema RIMAS (SGB).
    Ele navega poço por poço, baixa as séries históricas (CSV), calcula a variação do nível
    da água e a taxa de recarga/descarga anual, exportando tudo para um relatório Excel.
"""

import os
import time
import glob
import re
from datetime import datetime
import pandas as pd
import openpyxl

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

# --- CONFIGURAÇÕES GLOBAIS ---
DOWNLOAD_DIR = os.path.join(os.getcwd(), "pasta_downloads_rimas")

# Garante a criação do diretório de download
if not os.path.exists(DOWNLOAD_DIR):
    os.makedirs(DOWNLOAD_DIR)

def limpar_diretorio_download():
    """
    Remove arquivos existentes no diretório de download para evitar
    conflitos de leitura e garantir que o arquivo processado seja o atual.
    """
    for filename in os.listdir(DOWNLOAD_DIR):
        file_path = os.path.join(DOWNLOAD_DIR, filename)
        try:
            if os.path.isfile(file_path):
                os.remove(file_path)
        except Exception as e:
            print(f"[ERRO] Falha ao limpar diretório: {e}")

def configurar_driver():
    """
    Inicializa e configura a instância do Chrome WebDriver.
    Define as preferências de download automático e diretório padrão.
    """
    options = webdriver.ChromeOptions()
    options.add_argument('--start-maximized')
    
    prefs = {
        "download.default_directory": DOWNLOAD_DIR,
        "download.prompt_for_download": False,
        "directory_upgrade": True,
        "safebrowsing.enabled": True
    }
    options.add_experimental_option("prefs", prefs)
    
    service = Service(ChromeDriverManager().install())
    return webdriver.Chrome(service=service, options=options)

def converter_coordenada(texto_bruto):
    """
    Converte strings de coordenadas (formatos GGMMSS ou decimal texto)
    para float decimal. Assume coordenadas negativas (Hemisfério Sul/Oeste).
    """
    try:
        # Remove caracteres não numéricos, mantendo ponto e vírgula
        texto = re.sub(r'[^\d\.,-]', '', texto_bruto).replace(',', '.')
        
        # Lógica para formato GGMMSS (ex: 123000 -> 12° 30' 00")
        if len(texto) >= 6 and '.' not in texto:
            graus = float(texto[:2])
            minutos = float(texto[2:4])
            segundos = float(texto[4:6])
        else:
            # Lógica para formato espaçado (ex: 12 30 00)
            nums = re.findall(r"[\d\.]+", texto_bruto)
            if len(nums) >= 3:
                graus, minutos, segundos = float(nums[0]), float(nums[1]), float(nums[2])
            else:
                return 0.0
        
        decimal = graus + (minutos/60) + (segundos/3600)
        return -abs(decimal) 
    except Exception:
        return 0.0

def main():
    print("[INFO] Iniciando processo de coleta de dados (RIMAS)...")
    
    limpar_diretorio_download()
    driver = configurar_driver()
    wait = WebDriverWait(driver, 20)
    
    # Lista para armazenar dados preliminares (URL, UF)
    dados_pocos = []

    try:
        # --- ETAPA 1: Mapeamento da Lista Geral ---
        url_busca = 'https://rimasweb.sgb.gov.br/layout/resultado_busca_urucuia.php'
        driver.get(url_busca)
        print("[INFO] Carregando tabela de resultados...")
        time.sleep(5) 

        # Identifica linhas que contêm links para detalhes do poço
        linhas = driver.find_elements(By.XPATH, "//tr[.//a[contains(@href, 'detalhe.php')]]")
        print(f"[INFO] {len(linhas)} poços identificados. Iniciando mapeamento de UFs...")

        for linha in linhas:
            try:
                elem_link = linha.find_element(By.XPATH, ".//a[contains(@href, 'detalhe.php')]")
                url = elem_link.get_attribute('href')
                
                # Identifica UF na linha (Célula com 2 letras maiúsculas)
                celulas = linha.find_elements(By.TAG_NAME, "td")
                uf_encontrada = "N/A"
                for celula in celulas:
                    texto = celula.text.strip()
                    if len(texto) == 2 and texto.isupper() and texto.isalpha():
                        uf_encontrada = texto
                        break
                
                if url:
                    dados_pocos.append((url, uf_encontrada))
            except Exception:
                continue
        
        print(f"[INFO] Mapeamento concluído. Total de poços na fila: {len(dados_pocos)}")

    except Exception as e:
        print(f"[ERRO FATAL] Falha ao processar tabela inicial: {e}")
        driver.quit()
        return

    # --- ETAPA 2: Preparação do Relatório ---
    wb = openpyxl.Workbook()
    ws = wb.active
    ws.title = "Dados Consolidados"
    ws.append([
        'Nome', 'UF', 'Lat', 'Lon', 'Anos Monitorados', 
        'Nível Inicial (m)', 'Nível Final (m)', 'Variação (m)', 'Taxa (m/ano)'
    ])

    # --- ETAPA 3: Iteração e Extração ---
    total_pocos = len(dados_pocos)
    
    for i, (url, uf_correta) in enumerate(dados_pocos):
        try:
            print(f"\n[INFO] Processando poço {i+1}/{total_pocos} (UF: {uf_correta})...")
            limpar_diretorio_download()
            driver.get(url)

            # A. Extração de Metadados (Nome e Coordenadas)
            try:
                # Localiza elementos baseados no texto do label vizinho
                elem_nome = driver.find_element(By.XPATH, "//*[contains(text(), 'Nome:')]/following::td[1]")
                nome = elem_nome.text.strip()
                
                try:
                    elem_lat = driver.find_element(By.XPATH, "//*[contains(text(), 'Latitude')]/following::td[1]")
                    elem_lon = driver.find_element(By.XPATH, "//*[contains(text(), 'Longitude')]/following::td[1]")
                    lat = converter_coordenada(elem_lat.text)
                    lon = converter_coordenada(elem_lon.text)
                except:
                    lat, lon = 0.0, 0.0

                print(f"       -> Poço: {nome} | Coords: {lat:.4f}, {lon:.4f}")
                
            except Exception:
                nome = "Erro Leitura"
                lat, lon = 0.0, 0.0
                print("       [AVISO] Não foi possível ler o nome do poço.")

            # B. Navegação para aba 'Nível'
            try:
                xpath_aba = "//a[contains(., 'Nível')]"
                aba = wait.until(EC.presence_of_element_located((By.XPATH, xpath_aba)))
                
                # Scroll para o centro para evitar sobreposição de elementos
                driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", aba)
                time.sleep(1)
                driver.execute_script("arguments[0].click();", aba)
                time.sleep(4) # Aguarda carregamento AJAX da tabela
            except Exception:
                print("       [ERRO] Aba 'Nível' não encontrada ou não clicável. Pulando.")
                continue

            # C. Download do CSV
            try:
                # Scroll até o rodapé para renderizar o botão
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(2)
                
                xpath_btn = "//img[contains(@title, 'CSV')]"
                btn = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, xpath_btn)))
                
                driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", btn)
                time.sleep(1)
                driver.execute_script("arguments[0].click();", btn)
                
                print("       [INFO] Download iniciado. Aguardando conclusão...")
                time.sleep(10) 
            except Exception:
                print("       [AVISO] Botão CSV não encontrado. Provável ausência de dados.")
                continue

            # D. Processamento e Cálculo
            arquivos_csv = glob.glob(os.path.join(DOWNLOAD_DIR, "*.csv"))
            if arquivos_csv:
                try:
                    # Leitura do CSV (Encoding Latin-1 e Header na linha 2 padrão RIMAS)
                    df = pd.read_csv(arquivos_csv[0], encoding='latin-1', sep=';', decimal=',', header=2)
                    
                    # Identificação dinâmica de colunas
                    col_nivel = next((c for c in df.columns if "Nível da ág" in str(c)), None)
                    col_data = next((c for c in df.columns if "Data" in str(c)), None)

                    if col_nivel and col_data:
                        # Limpeza e conversão de tipos
                        df[col_nivel] = pd.to_numeric(df[col_nivel], errors='coerce')
                        df = df.dropna(subset=[col_nivel])

                        if not df.empty:
                            # Métricas Hidrogeológicas
                            n_ini = df[col_nivel].iloc[0]
                            n_fim = df[col_nivel].iloc[-1]
                            variacao = n_fim - n_ini 

                            # Cálculo Temporal
                            try:
                                d1 = pd.to_datetime(df[col_data].iloc[0], dayfirst=True)
                                d2 = pd.to_datetime(df[col_data].iloc[-1], dayfirst=True)
                                anos = abs((d2 - d1).days) / 365.25
                            except:
                                anos = 0

                            taxa = variacao / anos if anos > 0 else 0

                            # Persistência
                            ws.append([nome, uf_correta, lat, lon, round(anos, 2), n_ini, n_fim, round(variacao, 2), round(taxa, 4)])
                            print("       [SUCESSO] Dados processados e salvos.")
                        else:
                            print("       [AVISO] CSV vazio após limpeza de dados.")
                    else:
                        print("       [ERRO] Colunas esperadas não encontradas no arquivo.")
                
                except Exception as e:
                    print(f"       [ERRO] Falha na leitura do CSV: {e}")
            else:
                print("       [ERRO] Arquivo CSV não encontrado no diretório.")

            # Backup periódico
            if (i + 1) % 5 == 0:
                wb.save("relatorio_parcial_backup.xlsx")
                print("       [SYSTEM] Backup parcial realizado.")

        except Exception as e:
            print(f"[ERRO GERAL] Falha no poço {i+1}: {e}")
            continue

    # Finalização
    timestamp = datetime.now().strftime('%Y%m%d_%H%M')
    nome_arquivo_final = f"relatorio_rimas_final_{timestamp}.xlsx"
    wb.save(nome_arquivo_final)
    
    print("\n" + "="*50)
    print(f"[CONCLUÍDO] Processamento finalizado.")
    print(f"[ARQUIVO] Relatório salvo em: {nome_arquivo_final}")
    print("="*50)
    
    driver.quit()

if __name__ == "__main__":
    main()